In [1]:
# From here: https://stackoverflow.com/questions/49264194/import-py-file-in-another-directory-in-jupyter-notebook
import sys
import os

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
import copy

import src.mcmc as mcmc
import src.utils as utils
from src.ansatz import RBM
from src.model import Model
from src.mcmc import Walker
from src.hamiltonians import IsingHamiltonian, ReducedIsingHamiltonian
from config_parser import get_config_file

import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
time_list = []
for i in range(5, 10):
    H = IsingHamiltonian(i)
    time_list.append(utils.time_function(np.linalg.eig, H))
    

In [4]:
[print(f"Ising brute force: {time:8f}") for time in time_list]

Ising brute force: 0.000000
Ising brute force: 0.000000
Ising brute force: 0.000000
Ising brute force: 0.078125
Ising brute force: 0.218750


[None, None, None, None, None]

In [5]:
time_list = []
hidden_layer_size = 8

In [6]:

for i in range(5,16):
    H = ReducedIsingHamiltonian(i)
    
    b = utils.random_complex_array(i)  # Visible layer bias
    c = utils.random_complex_array(hidden_layer_size)  # Hidden layer bias
    W = utils.random_complex_matrix(i, hidden_layer_size)  # Visible - hidden weights
    
    rbm = RBM(visible_bias=b, hidden_bias=c,
              weights=W)  # Initializing RBM currently with random configuration and parameters

    walker = Walker(current_state=utils.random_binary_array(i))
    
    model = Model(rbm=rbm, walker=walker, hamiltonian=H)
    
    time_list.append(utils.time_function(model.estimate_energy))
    
    fd_plot_list_mc = model.gradient_descent(gradient_method='finite_difference', exact_dist=False)

Gradient descent step 1, energy: -0.12252593047039241
Gradient descent step 2, energy: -0.14228880972490235
Gradient descent step 3, energy: -0.08485403422752733
Gradient descent interrupted
Gradient descent step 1, energy: -0.6567518173935122
Gradient descent interrupted
Gradient descent step 1, energy: 0.21394994808136822
Gradient descent interrupted
Gradient descent step 1, energy: -0.16105124210127172
Gradient descent interrupted



KeyboardInterrupt



In [ ]:
[print(f"Ising MCMC: {time:8f}") for time in time_list]

In [ ]:
plt.plot(np.real(fd_plot_sparse_list), label='Sparse H')
plt.plot(np.real(fd_plot_full_list), label='Full H')


plt.axhline(y=min(np.linalg.eigvalsh(H_full)), color='red', linestyle='--', label='Ground State full')
plt.axhline(y=min(np.linalg.eigvalsh(H_sparse)), color='blue', linestyle='--', label='Ground State sparse')
plt.title(f"Sparse vs Full H gradent descent \n Visible:  {visible_layer_size}, hidden: {hidden_layer_size}")
plt.xlabel('Steps')
plt.ylabel('Energy')
plt.legend()
